Spaceship Titanic
=================

https://www.kaggle.com/competitions/spaceship-titanic/code?competitionId=34377

In [22]:
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


from credoai.lens import Lens
from credoai.artifacts import ClassificationModel, TabularData
from credoai.evaluators import ModelFairness, Performance


import warnings
warnings.filterwarnings('ignore')


Read datasets
-------------

In [23]:
ROOT_PATH = './input_data/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'

train_data = pd.read_csv(ROOT_PATH+TRAIN_FILE)
test_data = pd.read_csv(ROOT_PATH+TEST_FILE)
print(f'(train) Number of rows = {train_data.shape[0]} and Number of cols = {train_data.shape[1]}')
print(f'(test) Number of rows = {test_data.shape[0]} and Number of cols = {test_data.shape[1]}')

train_data

(train) Number of rows = 8693 and Number of cols = 14
(test) Number of rows = 4277 and Number of cols = 13


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


Analyze train data
------------------

In [24]:
def summary(df):
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    return summ

summary(train_data)

,dtypes,null,unique,min,median,max,mean,std
PassengerId,object,0,8693,0001_01,463001.0,9280_02,inf,NaN
HomePlanet,object,201,3,NaN,NaN,NaN,NaN,NaN
CryoSleep,object,217,2,False,0.0,True,0.358306,0.479531
Cabin,object,199,6560,NaN,NaN,NaN,NaN,NaN
Destination,object,182,3,NaN,NaN,NaN,NaN,NaN
Age,float64,179,80,0.0,27.0,79.0,28.827930,14.489021
VIP,object,203,2,False,0.0,True,0.023439,0.151303
RoomService,float64,181,1273,0.0,0.0,14327.0,224.687617,666.717663
FoodCourt,float64,183,1507,0.0,0.0,29813.0,458.077203,1611.489240
ShoppingMall,float64,208,1115,0.0,0.0,23492.0,173.729169,604.696458


Feature engineering
-------------------

Create a `FunctionTransformer` to remove Name and Cabin (not significative because too many different values), and split the PassengerId in the group id and person id.

In [25]:
def feature_engineering(data):
    return (
            data
            .assign(
                PassengerGGG = [x.split('_')[-0] for x in data['PassengerId']],
                PassengerPP = [x.split('_')[-1] for x in data['PassengerId']],
            )
            .drop(columns=['Name', 'PassengerId'])
        )

fe_eng = FunctionTransformer(feature_engineering)

fe_eng.fit_transform(train_data)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,PassengerGGG,PassengerPP
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0001,01
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0002,01
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0003,01
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0003,02
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0004,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,9276,01
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,9278,01
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,9279,01
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,9280,01


Create an `Imputer` to fill the missing values with the most frequent value in text columns, and with the mean in numeric ones.

In [26]:
imputer = ColumnTransformer(
    [
        (
            'label_imputer',
            SimpleImputer(strategy='most_frequent'),
            ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'PassengerGGG', 'PassengerPP']
        ),
        (
            'num_imputer',
            SimpleImputer(strategy='mean'),
            ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
        )
    ],
    verbose_feature_names_out=False,
    remainder='drop'
).set_output(transform='pandas')

imputer.fit_transform(fe_eng.fit_transform(train_data))

,HomePlanet,CryoSleep,Destination,VIP,PassengerGGG,PassengerPP,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,False,0001,01,39.0,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,False,0002,01,24.0,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,True,0003,01,58.0,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,False,0003,02,33.0,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,False,0004,01,16.0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,True,9276,01,41.0,0.0,6819.0,0.0,1643.0,74.0
8689,Earth,True,PSO J318.5-22,False,9278,01,18.0,0.0,0.0,0.0,0.0,0.0
8690,Earth,False,TRAPPIST-1e,False,9279,01,26.0,0.0,0.0,1872.0,1.0,0.0
8691,Europa,False,55 Cancri e,False,9280,01,32.0,0.0,1049.0,0.0,353.0,3235.0


In [27]:
scale_encode = ColumnTransformer(
    [
        (
            'std_scaler',
            StandardScaler(),
            ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
        ),
        (
            'minmax_scaler',
            MinMaxScaler(),
            ['PassengerGGG', 'PassengerPP']
        ),
        (
            'one_hot',
            OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist'),
            ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
        )
    ],
    verbose_feature_names_out=False
).set_output(transform='pandas')

scale_encode.fit_transform(imputer.fit_transform(fe_eng.fit_transform(train_data)))

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGGG,PassengerPP,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.709437,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,-0.336717,-0.175364,-0.281669,-0.248968,0.211505,-0.230194,0.000108,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2.034566,-0.275409,1.955616,-0.290817,5.694289,-0.225782,0.000216,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.290975,-0.340590,0.517406,0.330225,2.683471,-0.098708,0.000216,0.142857,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,-0.894666,0.118709,-0.243409,-0.038048,0.225732,-0.267258,0.000323,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.848924,-0.340590,3.989682,-0.290817,1.184286,-0.203720,0.999569,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
8689,-0.755179,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.999784,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8690,-0.197230,-0.340590,-0.287314,2.842851,-0.275774,-0.269023,0.999892,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
8691,0.221232,-0.340590,0.370637,-0.290817,0.037223,2.585740,1.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


Combine all transformers in a pipeline.

In [28]:
preproc = Pipeline(
    [
        ('fe_eng', fe_eng),
        ('imputer', imputer),
        ('scale_encode', scale_encode)
    ]
).set_output(transform='pandas')

preproc.fit_transform(train_data)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,PassengerGGG,PassengerPP,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.709437,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.000000,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,-0.336717,-0.175364,-0.281669,-0.248968,0.211505,-0.230194,0.000108,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,2.034566,-0.275409,1.955616,-0.290817,5.694289,-0.225782,0.000216,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.290975,-0.340590,0.517406,0.330225,2.683471,-0.098708,0.000216,0.142857,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,-0.894666,0.118709,-0.243409,-0.038048,0.225732,-0.267258,0.000323,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.848924,-0.340590,3.989682,-0.290817,1.184286,-0.203720,0.999569,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
8689,-0.755179,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023,0.999784,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
8690,-0.197230,-0.340590,-0.287314,2.842851,-0.275774,-0.269023,0.999892,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
8691,0.221232,-0.340590,0.370637,-0.290817,0.037223,2.585740,1.000000,0.000000,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


Create and train a model
------------------------

Create a model with the processing pipeline and one classifier.

In [29]:
model = Pipeline(
    [
        ('preproc', preproc),
        ('cla', LogisticRegression())
    ]
).set_output(transform='pandas')

Create a `GridSearchCV` to try many variants of the model, with different strategies and parameters, and find the combination with the best score.

In [30]:
param_grid = [
    {
        'preproc__imputer__num_imputer__strategy': ['mean', 'median'],
        'preproc__scale_encode__minmax_scaler__feature_range': [(0, 1), (-1, 1)],
        'cla': (LogisticRegression(),),
        'cla__C': [0.5, 1.0, 5.0],
        'cla__max_iter': [1000],
        'cla__class_weight': [None, 'balanced']
    },
    {
        'preproc__imputer__num_imputer__strategy': ['mean', 'median'],
        'preproc__scale_encode__minmax_scaler__feature_range': [(0, 1), (-1, 1)],
        'cla': (KNeighborsClassifier(),),
        'cla__n_neighbors': [3, 5, 7],
        'cla__weights': ['uniform', 'distance']
    },
    {
        'preproc__imputer__num_imputer__strategy': ['mean', 'median'],
        'preproc__scale_encode__minmax_scaler__feature_range': [(0, 1), (-1, 1)],
        'cla': (MLPClassifier(),),
        'cla__hidden_layer_sizes': [(20,), (25,), (30,)],
        'cla__activation': ['logistic', 'relu'],
        'cla__max_iter': [1500]
    },
    {
        'preproc__imputer__num_imputer__strategy': ['mean', 'median'],
        'preproc__scale_encode__minmax_scaler__feature_range': [(0, 1), (-1, 1)],
        'cla': (DecisionTreeClassifier(),),
        'cla__criterion': ['gini', 'entropy'],
        'cla__max_depth': [5, 8, 10]
    },
    {
        'preproc__imputer__num_imputer__strategy': ['mean', 'median'],
        'preproc__scale_encode__minmax_scaler__feature_range': [(0, 1), (-1, 1)],
        'cla': (RandomForestClassifier(),),
        'cla__n_estimators': [50, 100, 150],
        'cla__max_depth': [5, 8, 10]
    }
]   

gs = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,
    verbose=1,  # Set to 10 to print traces and know the % progress (very verbose)
    n_jobs=-1   # -1 uses all CPU cores; you can give a number > 0 to use that number of cores
)

Fit all variants and display the scores.

In [ ]:
gs.fit(train_data, train_data.Transported)

result = pd.DataFrame(gs.cv_results_).sort_values(by='rank_test_score').reset_index(drop=True)

result

Fitting 5 folds for each of 132 candidates, totalling 660 fits


Choose the optimal model

In [11]:
optimal_model = gs.best_estimator_

optimal_model

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('fe_eng',
                                  FunctionTransformer(func=<function feature_engineering at 0x7f2b677c20e0>)),
                                 ('imputer',
                                  ColumnTransformer(transformers=[('label_imputer',
                                                                   SimpleImputer(strategy='most_frequent'),
                                                                   ['HomePlanet',
                                                                    'CryoSleep',
                                                                    'Destination',
                                                                    'VIP',
                                                                    'PassengerGGG',
                                                                    'PassengerPP']),
                                                                  ('num_imputer',
                                                                   SimpleImputer(strategy='median')...
                                                                    'FoodCourt',
                                                                    'ShoppingMall',
                                                                    'Spa',
                                                                    'VRDeck']),
                                                                  ('minmax_scaler',
                                                                   MinMaxScaler(feature_range=(-1,
                                                                                               1)),
                                                                   ['PassengerGGG',
                                                                    'PassengerPP']),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                                 sparse_output=False),
                                                                   ['HomePlanet',
                                                                    'CryoSleep',
                                                                    'Destination',
                                                                    'VIP'])],
                                                    verbose_feature_names_out=False))])),
                ('cla',
                 MLPClassifier(hidden_layer_sizes=(25,), max_iter=1500))])

Use the model to predict the test data
----------------------------------

In [12]:
prediction = optimal_model.predict(test_data)

output = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Transported': prediction})
output.to_csv('submission.csv', index=False)

Save the optimal model
----------------------

In [13]:
with open('model.jlb', 'wb') as file:
    joblib.dump(optimal_model, file)

In [21]:
proc_data = preproc.fit_transform(train_data)
proc_data
model

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('fe_eng',
                                  FunctionTransformer(func=<function feature_engineering at 0x7f2b677c20e0>)),
                                 ('imputer',
                                  ColumnTransformer(transformers=[('label_imputer',
                                                                   SimpleImputer(strategy='most_frequent'),
                                                                   ['HomePlanet',
                                                                    'CryoSleep',
                                                                    'Destination',
                                                                    'VIP',
                                                                    'PassengerGGG',
                                                                    'PassengerPP']),
                                                                  ('num_imputer',
                                                                   SimpleImputer(),
                                                                   ['Age',
                                                                    'RoomServ...
                                  ColumnTransformer(transformers=[('std_scaler',
                                                                   StandardScaler(),
                                                                   ['Age',
                                                                    'RoomService',
                                                                    'FoodCourt',
                                                                    'ShoppingMall',
                                                                    'Spa',
                                                                    'VRDeck']),
                                                                  ('minmax_scaler',
                                                                   MinMaxScaler(),
                                                                   ['PassengerGGG',
                                                                    'PassengerPP']),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                                 sparse_output=False),
                                                                   ['HomePlanet',
                                                                    'CryoSleep',
                                                                    'Destination',
                                                                    'VIP'])],
                                                    verbose_feature_names_out=False))])),
                ('cla', LogisticRegression())])

In [18]:
credo_model = ClassificationModel(name="titanic_default_classifier", model_like=model)
credo_data = TabularData(
    name="titanic-default",
    X=train_data,
    y=train_data.Transported,
    sensitive_features=proc_data.Age,
)
lens = Lens(model=credo_model, assessment_data=credo_data, n_jobs=-1)
metrics = ['precision_score', 'recall_score', 'equal_opportunity']
lens.add(ModelFairness(metrics=metrics))
lens.add(Performance(metrics=metrics))
lens.run()

AttributeError: 'LogisticRegression' object has no attribute 'classes_'